In [1]:
import zipfile, os, re
import pandas as pd
import numpy as np 

# Read in the fooball data

In [2]:
_file = os.path.join('ncaa_football_scrapy','Data', 'Data.zip')
zf = zipfile.ZipFile(_file)
filenames = zf.namelist()

# Create Program History DF

In [125]:
history = pd.read_csv(zf.open('history.csv'))

In [126]:
history['year'] = history.apply(lambda x: int(x['Year'][:4]), axis = 1)

In [127]:
history_trimmed = history[['Team', "Wins", "Losses", "year", "WL"]]

In [129]:
history

,Year,HeadCoaches,Division,Conference,Wins,Losses,Ties,WL,Notes,Team,year
0,2017-18,Troy Calhoun,FBS,MWC,5.0,7.0,0.0,0.417,NaN,Air Force Falcons,2017
1,2016-17,Troy Calhoun,FBS,MWC,10.0,3.0,0.0,0.769,NaN,Air Force Falcons,2016
2,2015-16,Troy Calhoun,FBS,MWC,8.0,6.0,0.0,0.571,NaN,Air Force Falcons,2015
3,2014-15,Troy Calhoun,FBS,MWC,10.0,3.0,0.0,0.769,NaN,Air Force Falcons,2014
4,2013-14,Troy Calhoun,FBS,MWC,2.0,10.0,0.0,0.167,NaN,Air Force Falcons,2013
5,2012-13,Troy Calhoun,FBS,MWC,6.0,7.0,0.0,0.462,NaN,Air Force Falcons,2012
6,2011-12,Troy Calhoun,FBS,MWC,7.0,6.0,0.0,0.538,Military Bowl,Air Force Falcons,2011
7,2010-11,Troy Calhoun,FBS,MWC,9.0,4.0,0.0,0.692,Independence Bowl,Air Force Falcons,2010
8,2009-10,Troy Calhoun,FBS,MWC,8.0,5.0,0.0,0.615,Armed Forces Bowl,Air Force Falcons,2009
9,2008-09,Troy Calhoun,FBS,MWC,8.0,5.0,0.0,0.615,Armed Forces Bowl,Air Force Falcons,2008


In [6]:
team = 'Nebraska Cornhuskers, Huskers'
target_year = 2016
target_year_min = target_year - 3

In [7]:
history[(history.Team == team) & (history.year_new <= target_year) & (history.year_new >= target_year_min)
       ][['WL']].mean()

WL    0.6345
dtype: float64

In [8]:
history.head()

,Year,HeadCoaches,Division,Conference,Wins,Losses,Ties,WL,Notes,Team,year_new
0,2017-18,Troy Calhoun,FBS,MWC,5.0,7.0,0.0,0.417,NaN,Air Force Falcons,2017
1,2016-17,Troy Calhoun,FBS,MWC,10.0,3.0,0.0,0.769,NaN,Air Force Falcons,2016
2,2015-16,Troy Calhoun,FBS,MWC,8.0,6.0,0.0,0.571,NaN,Air Force Falcons,2015
3,2014-15,Troy Calhoun,FBS,MWC,10.0,3.0,0.0,0.769,NaN,Air Force Falcons,2014
4,2013-14,Troy Calhoun,FBS,MWC,2.0,10.0,0.0,0.167,NaN,Air Force Falcons,2013


# Create Coach DF 

In [114]:
coaches = pd.read_csv(zf.open('coaches.csv'))

In [115]:
coaches['year'] = coaches.apply(lambda x: int(x['Year'][:4]), axis = 1)

In [116]:
coach = 'Bob Diaco'
target_year = 2016

In [119]:
coach_record = coaches[(coaches.Name == coach) & (coaches.year <= target_year)][["Name","Wins","Losses","WL","year"]].groupby("Name").agg(['sum', 'count', 'mean'])

In [120]:
coach_record.columns = [x[0] + "_" + x[1]  for x in coach_record.columns.values]

In [121]:
coach_record.drop(['Wins_count', 'Wins_mean', 'Losses_count', 'Losses_mean', 'WL_sum', 'WL_count', 
                             'year_sum', 'year_mean'], axis=1, inplace = True)

In [122]:
coach_record['total_games'] = coach_record.Wins_sum + coach_record.Losses_sum
coach_record.reset_index(inplace = True)

In [123]:
coach_record.head()

,Name,Wins_sum,Losses_sum,WL_mean,year_count,total_games
0,Bob Diaco,11.0,26.0,0.293,3,37.0


In [124]:
coaches

,Year,Org,Division,Wins,Losses,Ties,WL,Notes,Name,year
0,2007-08,Air Force,FBS,9.0,4.0,0.0,0.692,NaN,Troy Calhoun,2007
1,2008-09,Air Force,FBS,8.0,5.0,0.0,0.615,NaN,Troy Calhoun,2008
2,2009-10,Air Force,FBS,8.0,5.0,0.0,0.615,NaN,Troy Calhoun,2009
3,2010-11,Air Force,FBS,9.0,4.0,0.0,0.692,NaN,Troy Calhoun,2010
4,2011-12,Air Force,FBS,7.0,6.0,0.0,0.538,NaN,Troy Calhoun,2011
5,2012-13,Air Force,FBS,6.0,7.0,0.0,0.462,NaN,Troy Calhoun,2012
6,2013-14,Air Force,FBS,2.0,10.0,0.0,0.167,NaN,Troy Calhoun,2013
7,2014-15,Air Force,FBS,10.0,3.0,0.0,0.769,NaN,Troy Calhoun,2014
8,2015-16,Air Force,FBS,8.0,6.0,0.0,0.571,NaN,Troy Calhoun,2015
9,2016-17,Air Force,FBS,10.0,3.0,0.0,0.769,NaN,Troy Calhoun,2016


# Create Roster DF

In [94]:
roster = pd.read_csv(zf.open('roster.csv'))

In [95]:
roster['year'] = roster.apply(lambda x: int(x['Year'][:4]), axis = 1)

In [96]:
def yrRecode(data): 
    yr = {'Fr':0, 'So':1, 'Jr':2, 'Sr':3}
    return yr[data]

In [97]:
roster_by_year = roster[['Team', 'Yr', 'year', 'GS']].groupby(['Team', 'year', 'Yr']).count().reset_index()

In [98]:
roster_games_started = roster[['Team', 'year', 'Yr', 'GS']].groupby(['Team', 'year', "Yr"]).mean().reset_index()

In [99]:
roster_games_played = roster[['Team', 'year', 'Yr', 'GP']].groupby(['Team', 'year', "Yr"]).mean().reset_index()

In [100]:
roster_final = pd.merge(roster_games_played, 
                        roster_games_started, 
                        left_on = ['Team', 'year', 'Yr'], 
                        right_on = ['Team', 'year', 'Yr'],
                        how = 'left'
                       )

In [101]:
roster_final2 = roster_final.set_index(['Team', 'year', 'Yr']).unstack().reset_index()

In [102]:
columns = [x[0] if x[0] in ['Team', 'year'] else x[0] + "_" + x[1] for x in roster_final2.columns.values]
roster_final2.columns = columns

In [103]:
roster_final2.head(3)

,Team,year,GP_Fr,GP_Jr,GP_So,GP_Sr,GS_Fr,GS_Jr,GS_So,GS_Sr
0,Air Force Falcons,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
1,Air Force Falcons,2014,1.000000,8.000000,6.363636,9.900000,0.000000,3.840000,1.575758,4.466667
2,Air Force Falcons,2015,1.800000,9.555556,6.714286,9.142857,0.000000,4.611111,0.885714,5.047619


In [28]:
def createvariables(data):
    """Create Opponent, Home, Win/Loss, Overtime, and Scores
    
    """
    regexs = {'WL': "[WL]",
              'team': ["\d+\s\-", "\d+"],
              'opponent_score':["\-\s\d+", '\d+'],
              'OT':["\(\d+OT\)", "\d+"], 
              'opponent':['[\w\s]+\.?\s\@\s\w+']
             }
    
    # Opponent 
    if '@' in data['Opponent']: 
        search = re.search(regexs['opponent'][0], data['Opponent']) 
        if search: 
            opponent = data['Opponent'].split("@")[0].strip()
            home = 0
        else: 
            opponent = data['Opponent'].replace("@","").strip()
            home = 0
    else: 
        opponent = data['Opponent']
        home = 1
    
    
    # Win/Loss
    WinLoss = re.search(regexs['WL'], data['Result']).group()
    
    # Overtime 
    Overtime = 0
    re_ot = re.search(regexs['OT'][0], data['Result'])
    if re_ot:
        Overtime = re.search(regexs['OT'][1], re_ot.group()).group()
        
    # Team Score
    team_score = None
    re_team = re.search(regexs['team'][0], data['Result'])
    if re_team:
        team_score = re.search(regexs['OT'][1], re_team.group()).group()
        
    # Opponent Score
    opponent_score = None
    re_opponent = re.search(regexs['opponent_score'][0], data['Result'])
    if re_opponent:
        opponent_score = re.search(regexs['OT'][1], re_opponent.group()).group()
        
    return pd.Series([opponent, home, WinLoss, Overtime, team_score, opponent_score])    

# Start merging

In [104]:
# Subset the gamebygame files
targetfiles = [file for file in filenames if 'gamebygame' in file ]

In [105]:
# Create a dict of dfs and create a multi-index 
dfs = {}
for f in targetfiles: 
    filename = f.split(".")[0]
    dfs[filename] = pd.read_csv(zf.open(f))
    dfs[filename]['Date'] = pd.to_datetime(dfs[filename]['Date'])
    dfs[filename].set_index(['Team', 'Date', 'OffenseDefense'], inplace = True)

In [106]:
# Concatenate the data together and drop duplicate column names 
master = pd.concat([v for k, v in dfs.items()], axis = 1)
master = master.loc[:, ~master.columns.duplicated()]

In [107]:
# Create new variables 
master[['opponent', 'home', 'WinLoss','Overtime', 'team_score', 'opponent_score']] = master.apply(lambda x: createvariables(x), axis = 1)

In [108]:
master[master.index.get_level_values('OffenseDefense') == 'Offense']

Opponent  \
Team              Date       OffenseDefense                                 
Air Force Falcons 2013-08-31 Offense                              Colgate   
                  2013-09-07 Offense                             Utah St.   
                  2013-09-13 Offense                          @ Boise St.   
                  2013-09-21 Offense                              Wyoming   
                  2013-09-28 Offense                             @ Nevada   
                  2013-10-05 Offense                               @ Navy   
                  2013-10-10 Offense                        San Diego St.   
                  2013-10-26 Offense                           Notre Dame   
                  2013-11-02 Offense                      Army West Point   
                  2013-11-08 Offense                         @ New Mexico   
                  2013-11-21 Offense                                 UNLV   
                  2013-11-30 Offense                       @ Colorado St.   
                  2014-08-30 Offense                         Nicholls St.   
                  2014-09-06 Offense                            @ Wyoming   
                  2014-09-13 Offense                        @ Georgia St.   
                  2014-09-27 Offense                            Boise St.   
                  2014-10-04 Offense                                 Navy   
                  2014-10-11 Offense                           @ Utah St.   
                  2014-10-18 Offense                           New Mexico   
                  2014-11-01 Offense                    @ Army West Point   
                  2014-11-08 Offense                               @ UNLV   
                  2014-11-15 Offense                               Nevada   
                  2014-11-21 Offense                      @ San Diego St.   
                  2014-11-28 Offense                         Colorado St.   
                  2014-12-20 Offense         Western Mich. @ Boise, Idaho   
                  2015-09-05 Offense                           Morgan St.   
                  2015-09-12 Offense                         San Jose St.   
                  2015-09-19 Offense                       @ Michigan St.   
                  2015-10-03 Offense                               @ Navy   
                  2015-10-10 Offense                              Wyoming   
...                                                                   ...   
Wyoming Cowboys   2014-11-01 Offense                         @ Fresno St.   
                  2014-11-07 Offense                             Utah St.   
                  2014-11-22 Offense                            Boise St.   
                  2014-11-29 Offense                         @ New Mexico   
                  2015-09-05 Offense                         North Dakota   
                  2015-09-12 Offense                        Eastern Mich.   
                  2015-09-19 Offense                     @ Washington St.   
                  2015-09-26 Offense                           New Mexico   
                  2015-10-03 Offense                    @ Appalachian St.   
                  2015-10-10 Offense                          @ Air Force   
                  2015-10-17 Offense                               Nevada   
                  2015-10-24 Offense                          @ Boise St.   
                  2015-10-30 Offense                           @ Utah St.   
                  2015-11-07 Offense                         Colorado St.   
                  2015-11-14 Offense                      @ San Diego St.   
                  2015-11-28 Offense                                 UNLV   
                  2016-09-03 Offense                        Northern Ill.   
                  2016-09-10 Offense                           @ Nebraska   
                  2016-09-17 Offense                             UC Davis   
                  2016-09-23 Offense                      @ E

In [109]:
master.reset_index(inplace = True)

In [110]:
master['year'] = master['Date'].dt.year

In [111]:
roster_final2.head()

,Team,year,GP_Fr,GP_Jr,GP_So,GP_Sr,GS_Fr,GS_Jr,GS_So,GS_Sr
0,Air Force Falcons,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
1,Air Force Falcons,2014,1.000000,8.000000,6.363636,9.900000,0.000000,3.840000,1.575758,4.466667
2,Air Force Falcons,2015,1.800000,9.555556,6.714286,9.142857,0.000000,4.611111,0.885714,5.047619
3,Air Force Falcons,2016,9.000000,6.545455,3.848485,10.484848,0.000000,2.454545,0.212121,5.939394
4,Akron Zips,2013,2.000000,7.592593,9.043478,10.090909,0.241379,2.851852,3.173913,4.772727


In [112]:
tmp = pd.merge(master,
               roster_final2, 
               left_on = ['Team', 'year'], 
               right_on = ['Team', 'year'],
               how = 'left'
              )

In [113]:
tmp2 = pd.merge(tmp, 
                
               )

,Team,Date,OffenseDefense,Opponent,Result,G,RushNetYards,ReceivingYards,IntRYds,PuntRetYds,...,opponent_score,year,GP_Fr,GP_Jr,GP_So,GP_Sr,GS_Fr,GS_Jr,GS_So,GS_Sr
0,Air Force Falcons,2013-08-31,Defense,Colgate,W 38 - 13,1/,157,143,NaN,NaN,...,13,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
1,Air Force Falcons,2013-08-31,Offense,Colgate,W 38 - 13,1/,409,72,NaN,14,...,13,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
2,Air Force Falcons,2013-09-07,Defense,Utah St.,L 20 - 52,1/,217,360,NaN,-5,...,52,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
3,Air Force Falcons,2013-09-07,Offense,Utah St.,L 20 - 52,1/,162,108,NaN,NaN,...,52,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
4,Air Force Falcons,2013-09-13,Defense,@ Boise St.,L 20 - 42,1/,229,304,NaN,NaN,...,42,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
5,Air Force Falcons,2013-09-13,Offense,@ Boise St.,L 20 - 42,1/,188,99,17,NaN,...,42,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
6,Air Force Falcons,2013-09-21,Defense,Wyoming,L 23 - 56,1/,249,373,NaN,NaN,...,56,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
7,Air Force Falcons,2013-09-21,Offense,Wyoming,L 23 - 56,1/,346,127,NaN,-8,...,56,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
8,Air Force Falcons,2013-09-28,Defense,@ Nevada,L 42 - 45,1/,210,389,6,19,...,45,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
9,Air Force Falcons,2013-09-28,Offense,@ Nevada,L 42 - 45,1/,375,78,21,NaN,...,45,2013,5.133333,7.783784,6.956522,9.000000,0.666667,2.459459,3.521739,4.411765
